<a href="https://colab.research.google.com/github/sandeep92134/The-Data-Science-Workshop-By-Packt/blob/master/module%2017/Exercise17.02%3A%20Creating%20New%20Features%20Using%20Deep%20Feature%20Synthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Creating Entities and relationships**

In [1]:
# Defining the path to the 
file_url = 'https://raw.githubusercontent.com/sandeep92134/The-Data-Science-Workshop-By-Packt/master/module%2017/datasets/bank-full.csv'

In [2]:
# Loading data using pandas
import pandas as pd
bankData = pd.read_csv(file_url,sep=";")
bankData.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
# Removing the target variable
Y = bankData.pop('y')

In [4]:
# Creating the Ids for Demographic Entity
bankData['custID'] = bankData.index.values

bankData['custID'] = 'cust' + bankData['custID'].astype(str)

In [5]:
# Creating AssetId
bankData['AssetId'] = 0
bankData.loc[bankData.housing == 'yes','AssetId']= 1

In [6]:
# Creating LoanId
bankData['LoanId'] = 0
bankData.loc[bankData.loan == 'yes','LoanId']= 1

In [7]:
# Creating Financial behaviour ID
bankData['FinbehId'] = 0
bankData.loc[bankData.default == 'yes','FinbehId']= 1

In [8]:
# Importing necessary libraries
import featuretools as ft
import numpy as np

In [9]:
# creating the entity set 'Bankentities'
Bankentities = ft.EntitySet(id = 'Bank')

In [10]:
# Mapping a dataframe to the entityset to form the parent entity
Bankentities.entity_from_dataframe(entity_id = 'Demographic Data', dataframe = bankData, index = 'custID')

Entityset: Bank
  Entities:
    Demographic Data [Rows: 45211, Columns: 20]
  Relationships:
    No relationships

In [11]:
# Mapping Assets and setting the relationship
Bankentities.normalize_entity(base_entity_id='Demographic Data', new_entity_id='Assets', index = 'AssetId', 
additional_variables = ['housing'])

Entityset: Bank
  Entities:
    Demographic Data [Rows: 45211, Columns: 19]
    Assets [Rows: 2, Columns: 2]
  Relationships:
    Demographic Data.AssetId -> Assets.AssetId

In [12]:
# Mapping Loans and Financial behavior entities
Bankentities.normalize_entity(base_entity_id='Demographic Data', new_entity_id='Liability', index = 'LoanId', 
additional_variables = ['loan'])

Bankentities.normalize_entity(base_entity_id='Demographic Data', new_entity_id='FinBehaviour', index = 'FinbehId', 
additional_variables = ['default'])

Entityset: Bank
  Entities:
    Demographic Data [Rows: 45211, Columns: 17]
    Assets [Rows: 2, Columns: 2]
    Liability [Rows: 2, Columns: 2]
    FinBehaviour [Rows: 2, Columns: 2]
  Relationships:
    Demographic Data.AssetId -> Assets.AssetId
    Demographic Data.LoanId -> Liability.LoanId
    Demographic Data.FinbehId -> FinBehaviour.FinbehId

**Feature Engineering**

In [13]:
# Creating feature sets using Deep Feature Synthesis
feature_set, feature_names = ft.dfs(entityset=Bankentities, 
target_entity = 'Demographic Data', 
max_depth = 2, 
verbose = 1, 
n_jobs = 1)

Built 196 features
Elapsed: 00:01 | Progress: 100%|██████████


In [14]:
# Reindexing the feature_set
feature_set = feature_set.reindex(index=bankData['custID'])
feature_set = feature_set.reset_index()

In [15]:
# Verifying the shape of the features and original bank data
print(feature_set.shape)
print(bankData.shape)

(45211, 197)
(45211, 20)


In [16]:
# Printing head of the feature set
feature_set.head()

,custID,age,job,marital,education,balance,contact,day,month,duration,campaign,pdays,previous,poutcome,AssetId,LoanId,FinbehId,Assets.housing,Liability.loan,FinBehaviour.default,Assets.COUNT(Demographic Data),Assets.MAX(Demographic Data.age),Assets.MAX(Demographic Data.balance),Assets.MAX(Demographic Data.campaign),Assets.MAX(Demographic Data.day),Assets.MAX(Demographic Data.duration),Assets.MAX(Demographic Data.pdays),Assets.MAX(Demographic Data.previous),Assets.MEAN(Demographic Data.age),Assets.MEAN(Demographic Data.balance),Assets.MEAN(Demographic Data.campaign),Assets.MEAN(Demographic Data.day),Assets.MEAN(Demographic Data.duration),Assets.MEAN(Demographic Data.pdays),Assets.MEAN(Demographic Data.previous),Assets.MIN(Demographic Data.age),Assets.MIN(Demographic Data.balance),Assets.MIN(Demographic Data.campaign),Assets.MIN(Demographic Data.day),Assets.MIN(Demographic Data.duration),...,FinBehaviour.MIN(Demographic Data.duration),FinBehaviour.MIN(Demographic Data.pdays),FinBehaviour.MIN(Demographic Data.previous),FinBehaviour.MODE(Demographic Data.AssetId),FinBehaviour.MODE(Demographic Data.LoanId),FinBehaviour.MODE(Demographic Data.contact),FinBehaviour.MODE(Demographic Data.education),FinBehaviour.MODE(Demographic Data.job),FinBehaviour.MODE(Demographic Data.marital),FinBehaviour.MODE(Demographic Data.month),FinBehaviour.MODE(Demographic Data.poutcome),FinBehaviour.NUM_UNIQUE(Demographic Data.AssetId),FinBehaviour.NUM_UNIQUE(Demographic Data.LoanId),FinBehaviour.NUM_UNIQUE(Demographic Data.contact),FinBehaviour.NUM_UNIQUE(Demographic Data.education),FinBehaviour.NUM_UNIQUE(Demographic Data.job),FinBehaviour.NUM_UNIQUE(Demographic Data.marital),FinBehaviour.NUM_UNIQUE(Demographic Data.month),FinBehaviour.NUM_UNIQUE(Demographic Data.poutcome),FinBehaviour.SKEW(Demographic Data.age),FinBehaviour.SKEW(Demographic Data.balance),FinBehaviour.SKEW(Demographic Data.campaign),FinBehaviour.SKEW(Demographic Data.day),FinBehaviour.SKEW(Demographic Data.duration),FinBehaviour.SKEW(Demographic Data.pdays),FinBehaviour.SKEW(Demographic Data.previous),FinBehaviour.STD(Demographic Data.age),FinBehaviour.STD(Demographic Data.balance),FinBehaviour.STD(Demographic Data.campaign),FinBehaviour.STD(Demographic Data.day),FinBehaviour.STD(Demographic Data.duration),FinBehaviour.STD(Demographic Data.pdays),FinBehaviour.STD(Demographic Data.previous),FinBehaviour.SUM(Demographic Data.age),FinBehaviour.SUM(Demographic Data.balance),FinBehaviour.SUM(Demographic Data.campaign),FinBehaviour.SUM(Demographic Data.day),FinBehaviour.SUM(Demographic Data.duration),FinBehaviour.SUM(Demographic Data.pdays),FinBehaviour.SUM(Demographic Data.previous)
0,cust0,58,management,married,tertiary,2143,unknown,5,may,261,1,-1,0,unknown,1,0,0,yes,no,no,25130,78,58544,63,31,4918,854,275,39.175288,1175.103064,2.698488,15.598249,259.331476,51.312495,0.656665,20,-4057,1,1,0,...,0,-1,0,1,0,cellular,secondary,blue-collar,married,may,unknown,2,2,3,4,12,3,12,4,0.687031,8.334719,4.926324,0.094405,3.152469,2.600138,41.986484,10.638882,3063.275425,3.087038,8.317696,258.239396,100.50463,2.313596,1818546,61701846,122390,701270,11476932,1802679,26018
1,cust1,44,technician,single,secondary,29,unknown,5,may,151,1,-1,0,unknown,1,0,0,yes,no,no,25130,78,58544,63,31,4918,854,275,39.175288,1175.103064,2.698488,15.598249,259.331476,51.312495,0.656665,20,-4057,1,1,0,...,0,-1,0,1,0,cellular,secondary,blue-collar,married,may,unknown,2,2,3,4,12,3,12,4,0.687031,8.334719,4.926324,0.094405,3.152469,2.600138,41.986484,10.638882,3063.275425,3.087038,8.317696,258.239396,100.50463,2.313596,1818546,61701846,122390,701270,11476932,1802679,26018
2,cust2,33,entrepreneur,married,secondary,2,unknown,5,may,76,1,-1,0,unknown,1,1,0,yes,yes,no,25130,78,58544,63,31,4918,854,275,39.175288,1175.103064,2.698488,15.598249,259.331476,51.312495,0.656665,20,-4057,1,1,0,...,0,-1,0,1,0,cellular,secondary,blue-collar,married,may,unknown,2,2,3,4,12,3,12,4,0.687031,8.334719,4.926324,0.094405,3.152469,2.600138,41.986484,10.6388

In [17]:
# Verifying the features for Assets.SUM(Demographic Data.balance)
bankData.groupby('AssetId')['balance'].agg('sum')

AssetId
0    32059342
1    29530340
Name: balance, dtype: int64

In [18]:
# Printing the list of all features
feature_names

[<Feature: age>,
 <Feature: job>,
 <Feature: marital>,
 <Feature: education>,
 <Feature: balance>,
 <Feature: contact>,
 <Feature: day>,
 <Feature: month>,
 <Feature: duration>,
 <Feature: campaign>,
 <Feature: pdays>,
 <Feature: previous>,
 <Feature: poutcome>,
 <Feature: AssetId>,
 <Feature: LoanId>,
 <Feature: FinbehId>,
 <Feature: Assets.housing>,
 <Feature: Liability.loan>,
 <Feature: FinBehaviour.default>,
 <Feature: Assets.COUNT(Demographic Data)>,
 <Feature: Assets.MAX(Demographic Data.age)>,
 <Feature: Assets.MAX(Demographic Data.balance)>,
 <Feature: Assets.MAX(Demographic Data.campaign)>,
 <Feature: Assets.MAX(Demographic Data.day)>,
 <Feature: Assets.MAX(Demographic Data.duration)>,
 <Feature: Assets.MAX(Demographic Data.pdays)>,
 <Feature: Assets.MAX(Demographic Data.previous)>,
 <Feature: Assets.MEAN(Demographic Data.age)>,
 <Feature: Assets.MEAN(Demographic Data.balance)>,
 <Feature: Assets.MEAN(Demographic Data.campaign)>,
 <Feature: Assets.MEAN(Demographic Data.day)>,


**Configuring the primitives**

In [19]:
# Creating aggregation and transformation primitives
aggPrimitives=[
        'std', 'min', 'max', 'mean', 
         'last', 'count'
        
]
tranPrimitives=[
        'percentile', 
         'subtract', 'divide']

In [21]:
# Displaying the feature set 
feature_set.head()

,custID,age,job,marital,education,balance,contact,day,month,duration,campaign,pdays,previous,poutcome,AssetId,LoanId,FinbehId,Assets.housing,Liability.loan,FinBehaviour.default,Assets.COUNT(Demographic Data),Assets.MAX(Demographic Data.age),Assets.MAX(Demographic Data.balance),Assets.MAX(Demographic Data.campaign),Assets.MAX(Demographic Data.day),Assets.MAX(Demographic Data.duration),Assets.MAX(Demographic Data.pdays),Assets.MAX(Demographic Data.previous),Assets.MEAN(Demographic Data.age),Assets.MEAN(Demographic Data.balance),Assets.MEAN(Demographic Data.campaign),Assets.MEAN(Demographic Data.day),Assets.MEAN(Demographic Data.duration),Assets.MEAN(Demographic Data.pdays),Assets.MEAN(Demographic Data.previous),Assets.MIN(Demographic Data.age),Assets.MIN(Demographic Data.balance),Assets.MIN(Demographic Data.campaign),Assets.MIN(Demographic Data.day),Assets.MIN(Demographic Data.duration),...,FinBehaviour.MIN(Demographic Data.duration),FinBehaviour.MIN(Demographic Data.pdays),FinBehaviour.MIN(Demographic Data.previous),FinBehaviour.MODE(Demographic Data.AssetId),FinBehaviour.MODE(Demographic Data.LoanId),FinBehaviour.MODE(Demographic Data.contact),FinBehaviour.MODE(Demographic Data.education),FinBehaviour.MODE(Demographic Data.job),FinBehaviour.MODE(Demographic Data.marital),FinBehaviour.MODE(Demographic Data.month),FinBehaviour.MODE(Demographic Data.poutcome),FinBehaviour.NUM_UNIQUE(Demographic Data.AssetId),FinBehaviour.NUM_UNIQUE(Demographic Data.LoanId),FinBehaviour.NUM_UNIQUE(Demographic Data.contact),FinBehaviour.NUM_UNIQUE(Demographic Data.education),FinBehaviour.NUM_UNIQUE(Demographic Data.job),FinBehaviour.NUM_UNIQUE(Demographic Data.marital),FinBehaviour.NUM_UNIQUE(Demographic Data.month),FinBehaviour.NUM_UNIQUE(Demographic Data.poutcome),FinBehaviour.SKEW(Demographic Data.age),FinBehaviour.SKEW(Demographic Data.balance),FinBehaviour.SKEW(Demographic Data.campaign),FinBehaviour.SKEW(Demographic Data.day),FinBehaviour.SKEW(Demographic Data.duration),FinBehaviour.SKEW(Demographic Data.pdays),FinBehaviour.SKEW(Demographic Data.previous),FinBehaviour.STD(Demographic Data.age),FinBehaviour.STD(Demographic Data.balance),FinBehaviour.STD(Demographic Data.campaign),FinBehaviour.STD(Demographic Data.day),FinBehaviour.STD(Demographic Data.duration),FinBehaviour.STD(Demographic Data.pdays),FinBehaviour.STD(Demographic Data.previous),FinBehaviour.SUM(Demographic Data.age),FinBehaviour.SUM(Demographic Data.balance),FinBehaviour.SUM(Demographic Data.campaign),FinBehaviour.SUM(Demographic Data.day),FinBehaviour.SUM(Demographic Data.duration),FinBehaviour.SUM(Demographic Data.pdays),FinBehaviour.SUM(Demographic Data.previous)
0,cust0,58,management,married,tertiary,2143,unknown,5,may,261,1,-1,0,unknown,1,0,0,yes,no,no,25130,78,58544,63,31,4918,854,275,39.175288,1175.103064,2.698488,15.598249,259.331476,51.312495,0.656665,20,-4057,1,1,0,...,0,-1,0,1,0,cellular,secondary,blue-collar,married,may,unknown,2,2,3,4,12,3,12,4,0.687031,8.334719,4.926324,0.094405,3.152469,2.600138,41.986484,10.638882,3063.275425,3.087038,8.317696,258.239396,100.50463,2.313596,1818546,61701846,122390,701270,11476932,1802679,26018
1,cust1,44,technician,single,secondary,29,unknown,5,may,151,1,-1,0,unknown,1,0,0,yes,no,no,25130,78,58544,63,31,4918,854,275,39.175288,1175.103064,2.698488,15.598249,259.331476,51.312495,0.656665,20,-4057,1,1,0,...,0,-1,0,1,0,cellular,secondary,blue-collar,married,may,unknown,2,2,3,4,12,3,12,4,0.687031,8.334719,4.926324,0.094405,3.152469,2.600138,41.986484,10.638882,3063.275425,3.087038,8.317696,258.239396,100.50463,2.313596,1818546,61701846,122390,701270,11476932,1802679,26018
2,cust2,33,entrepreneur,married,secondary,2,unknown,5,may,76,1,-1,0,unknown,1,1,0,yes,yes,no,25130,78,58544,63,31,4918,854,275,39.175288,1175.103064,2.698488,15.598249,259.331476,51.312495,0.656665,20,-4057,1,1,0,...,0,-1,0,1,0,cellular,secondary,blue-collar,married,may,unknown,2,2,3,4,12,3,12,4,0.687031,8.334719,4.926324,0.094405,3.152469,2.600138,41.986484,10.6388